In [58]:
import csv
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

# set default
pio.templates.default = "plotly_dark"

In [52]:
# read data
df = pd.read_csv('All_States_GE.csv')
df

ed_map = {
    "Others": 0,
    "Illiterate": 1,
    "Literate": 2,
    "5th Pass": 3,
    "8th Pass": 4,
    "10th Pass": 5,
    "12th Pass": 6,
    "Graduate": 7,
    "Graduate Professional": 8,
    "Post Graduate": 9,
    "Doctorate": 10
}

C:\Users\Rahul\AppData\Local\Temp\ipykernel_8592\2377711653.py:2: DtypeWarning: Columns (17,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('All_States_GE.csv')


In [53]:
# drop rows where sex is NaN
fdf = df.dropna(subset=["Sex", "Votes"])
# Convert M and F to int values
sex_mapping = {"M": 0, "F": 1}
fdf["SexInt"] = fdf["Sex"].map(sex_mapping)
fdf["EducationInt"] = fdf["MyNeta_education"].map(ed_map)
fdf

C:\Users\Rahul\AppData\Local\Temp\ipykernel_8592\565342313.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf["SexInt"] = fdf["Sex"].map(sex_mapping)
C:\Users\Rahul\AppData\Local\Temp\ipykernel_8592\565342313.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf["EducationInt"] = fdf["MyNeta_education"].map(ed_map)


,State_Name,Assembly_No,Constituency_No,Year,month,Poll_No,DelimID,Position,Candidate,Sex,...,Incumbent,Recontest,MyNeta_education,TCPD_Prof_Main,TCPD_Prof_Main_Desc,TCPD_Prof_Second,TCPD_Prof_Second_Desc,Election_Type,SexInt,EducationInt
0,Andaman_&_Nicobar_Islands,17,1,2019,4.0,0,4,1,KULDEEP RAI SHARMA,M,...,False,True,Graduate Professional,Business,NaN,Social Work,NaN,Lok Sabha Election (GE),0.0,8.0
1,Andaman_&_Nicobar_Islands,17,1,2019,4.0,0,4,2,VISHAL JOLLY,M,...,False,False,Graduate Professional,Liberal Profession or Professional,Lawyer,NaN,NaN,Lok Sabha Election (GE),0.0,8.0
2,Andaman_&_Nicobar_Islands,17,1,2019,4.0,0,4,3,PARITOSH KUMAR HALDAR,M,...,False,False,Post Graduate,Agriculture,NaN,NaN,NaN,Lok Sabha Election (GE),0.0,9.0
3,Andaman_&_Nicobar_Islands,17,1,2019,4.0,0,4,4,SANJAY MESHACK,M,...,False,True,12th Pass,Business,NaN,Politics,Municipality Member,Lok Sabha Election (GE),0.0,6.0
4,Andaman_&_Nicobar_Islands,17,1,2019,4.0,0,4,5,PRAKASH MINJ,M,...,False,False,Post Graduate,Social Work,NaN,NaN,NaN,Lok Sabha Election (GE),0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91646,Uttar_Pradesh,3,56,1964,NaN,1,1,1,V.L.PANDIT,F,...,False,False,NaN,NaN,NaN,NaN,NaN,Lok Sabha Election (GE),1.0,NaN
91651,Andhra_Pradesh,3,40,1965,NaN,1,1,1,R.S.REDDY,M,...,False,False,NaN,NaN,NaN,NaN,NaN,Lok Sabha Election (GE),0.0,NaN
91657,Mysore,3,19,1965,NaN,1,1,1,MALIMARIYAPPA,M,...,False,False,NaN,NaN,NaN,NaN,NaN,Lok Sabha Election (GE),0.0,NaN
91658,Mysore,3,19,1965,NaN,1,1,2,K.V.SUBRAMANYASWAMY,M,...,False,True,NaN,NaN,NaN,NaN,NaN,Lok Sabha Election (GE),0.0,NaN


In [54]:
# if someone has more votes, they are more likely to contest again. And if someone is contesting again, they are more likely to have more votes.
fdf["Votes"].corr(fdf["Contested"])

0.3741060992746316

In [55]:
# avg of votes for women and men sectioned by Election Number
elNumMFDict = {}
for i in range(len(fdf)):
    elNum = fdf.iloc[i]["Assembly_No"]
    if elNum not in elNumMFDict:
        elNumMFDict[elNum] = [0, 0, 0, 0]

    if fdf.iloc[i]["SexInt"] == 0:
        elNumMFDict[elNum][0] += fdf.iloc[i]["Votes"]
        elNumMFDict[elNum][1] += 1
    else:
        elNumMFDict[elNum][2] += fdf.iloc[i]["Votes"]
        elNumMFDict[elNum][3] += 1


In [62]:
avgDict = {}
for elNum in elNumMFDict:
    avgDict[elNum] = [elNumMFDict[elNum][0] / elNumMFDict[elNum][1],
                      elNumMFDict[elNum][2] / elNumMFDict[elNum][3]]
    
avgDict

# plot 2 line graphs
keys = list(avgDict.keys())
line1 = [point[0] for point in avgDict.values()]
line2 = [point[1] for point in avgDict.values()]

fig = go.Figure()

fig.add_trace(go.Scatter(x=keys, y=line1, mode='lines', name='Male'))
fig.add_trace(go.Scatter(x=keys, y=line2, mode='lines', name='Female'))

fig.update_layout(title='Male and Female Average Votes',
                  xaxis_title='X Axis',
                  yaxis_title='Values',
                  hovermode='x')

fig.show()
